# Manipulation des données de la base Neo4J d'Euclid
Ce notebook a pour but d'extraire les données Neo4J du projet Euclid (ou d'une base de données test), et d'y appliquer un algorithme de clustering.

#### Ici, nous utilisons comme méthode de clustering la *méthode Louvain*.

In [1]:
from IPython.display import SVG

import numpy as np
import pickle as pkl

from neo4j import GraphDatabase

from sknetwork.clustering import Louvain, modularity
from sknetwork.visualization import svg_digraph

In [2]:
def remove_names_NS(node_names, namespaces):
    """
    Fonction pour retirer dans la requête les noeuds du namespace
    qui commencent par les chaînes contenues dans la liste
    """
    out_cond = 'NOT ('
    out_cond += ") AND NOT (".join(" OR ".join(f'{node_name}.fullname =~ "{ns}.*"' for ns in namespaces) for node_name in node_names)
    out_cond += ')'
    
    return out_cond

### Si test 

In [ ]:
nodeType, relType, version, use_weights = "Node", "LINKED_TO", "V1", False

### Sinon 

In [3]:
#Type des noeuds et des relations du sous-graphe voulu
nodeType, relType = "Type", "USE_TYPE"

#Si on veut combiner des relations (CONTAINS et USE_NS par exemple)
#nodeType = "CONTAINS|USE_NS"

#Utilisation de poids, sinon une relation vaut 1
#Le sous-graphe avec la relation CONTAINS est un arbre, on ne sert donc pas de la variable pour elle
use_weights = True

version = "V1"

 Namespaces à enlever selon la version :
 
* V1 -> Aucune restriction
* V2 -> Pas de namespace lié à la DPD
* V3 -> V2 + pas de namespace lié à PRO
* V4 -> V3 + pas de namespace lié à INTERFACES

In [4]:
str_weight = 'W' if use_weights and relType in  {'USE_NS', 'USE_TYPE'} else ''

In [5]:
#True si les types viennent d'un même namespace
#False sinon
type_same_ns = False

cond = ""

if version == "V1":
    remove_ns = []
elif version == "V2":
    remove_ns = ['dpd']
elif version == "V3":
    remove_ns = ['dpd','pro']
elif version == "V4":
    remove_ns = ['dpd','pro','interfaces']
    
if version not in {"V0","V1"}:
    if nodeType == 'Type':
        nodeNames = ['ns'] if type_same_ns else ['ns','ns2'] 
    else:
        nodeNames = ['n1','n2']
    
    cond = f"WHERE {remove_names_NS(nodeNames,remove_ns)}"

In [6]:
def query_start(d="",same_ns=True):
    """
    Décrit le début de la requête Cypher sur le sous-graphe Neo4J désiré
    
    d: Direction de la relation
    same_ns: indique si les noeuds de type Type doivent faire partie du même namespace
    """
    q = f'(n1:{nodeType}){"<-" if d == "l" else "-"}[r:{relType}]{"->" if d == "r" else "-"}(n2:{nodeType})'  
    
    if nodeType == "Type":
        q = f"(ns:Namespace)<-[:DECLARED_IN]-{q}-[:DECLARED_IN]->({'ns' if same_ns else 'ns2:Namespace'})"
        
    q = f'MATCH {q} {cond}'
    
    return q

In [7]:
#Nom du paramètre de poids selon la relation
q_poids = lambda: "n_times" if relType == "USE_TYPE" else "nb_use"

#Nom du paramètre de nom selon la relation
q_name = lambda: "fullname" if nodeType == "Namespace" else "name"

#Sous forme de fonction lambda pour pouvoir changer facilement de relations

In [8]:
query_start(same_ns=type_same_ns), q_poids(), q_name(), cond

('MATCH (ns:Namespace)<-[:DECLARED_IN]-(n1:Type)-[r:USE_TYPE]-(n2:Type)-[:DECLARED_IN]->(ns2:Namespace) ',
 'n_times',
 'name',
 '')

### Base Euclid

In [9]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","euclid"))
session = driver.session()

##### Requête pour obtenir le nombre de noeuds du graphe (ne compte pas les noeuds liés à aucune relation)

In [10]:
query_nbnoeuds = f'{query_start(same_ns=type_same_ns)} RETURN COUNT(DISTINCT n1) as nb'

### Base de test

In [ ]:
driverTest = GraphDatabase.driver("bolt://localhost:11005", auth=("neo4j","0"))
session = driverTest.session()

##### Requête pour obtenir le nombre de noeuds du graphe (ne compte pas les noeuds liés à aucune relation)

In [ ]:
query_nbnoeuds = "MATCH (:Node) RETURN COUNT(*) as nb"

-------------

#### Nombre de noeuds
Ne compte pas les noeuds qui ne sont pas liés par une relation `relType`

In [ ]:
res = session.run(query_nbnoeuds)
for r in res: nbnoeuds = r["nb"]

In [ ]:
nbnoeuds

------------------

#### Récupération du poids maximal de la relation concernée

In [ ]:
query = f'{query_start(same_ns=type_same_ns)} RETURN MAX(TOINTEGER(r.{q_poids()})) AS max_poids'
res = session.run(query)

In [ ]:
for r in res: max_poids = r['max_poids']

In [ ]:
max_poids

### Calcul de la matrice laplacienne L

In [ ]:
ids_dict = dict()

def new_node_id(id):
    """
    Fonction pour générer de nouveaux IDs
    """
    id_num = len(ids_dict)
    if id not in ids_dict:
        ids_dict[id] = id_num
        id_num += 1
    return ids_dict[id]

def calc_A(query,use_weights=False,norm=True):
    """
    Calcule la matrice laplacienne L et retourne aussi
    les matrices des degrés D et la matrice d'adjacence A
    mise à jour lors du calcul
    """
    A = np.zeros((nbnoeuds,nbnoeuds))

    res = session.run(query)
    i=0
    for r in res:
        id1, id2 = new_node_id(r["id1"]), new_node_id(r["id2"])

        if use_weights and relType in {"USE_NS","USE_TYPE"}:
            poids = (r["poids"]*(0.1 if r['opt'] == "yes" else 1)+1)/(max_poids+1)
        else:
            poids = 1

        #Ajout dans la matrice d'adjacence
        A[id1,id2] = poids
        i += 1
        

    #On travaille sur le graphe orienté
    #la matrice n'est donc pas symétrique
    assert not np.array_equal(A,A.T)
    
    return A

----------

#### Calcul de la matrice d'adjacence 

In [ ]:
query = f'{query_start("r",type_same_ns)} RETURN ID(n1) AS id1, ID(n2) AS id2, TOINTEGER(r.{q_poids()}) AS poids, r.optional as opt'
A = calc_A(query,use_weights=True)

#### Tentative de combinaison des résultats des relations **USE_NS** et **CONTAINS**

In [ ]:
relType = "USE_NS"
query = f'{query_start("r",type_same_ns)} RETURN ID(n1) AS id1, ID(n2) AS id2, TOINTEGER(r.{q_poids()}) AS poids, r.optional as opt'
A = calc_A(query,use_weights=True)

relType = "CONTAINS"
query = f'{query_start(same_ns=type_same_ns)} RETURN ID(n1) AS id1, ID(n2) AS id2, TOINTEGER(r.{q_poids()}) AS poids'
A2 = calc_A(query)

In [ ]:
wA = 0.75
wA2 = 1-wA

A = wA*A + wA2*A2

------------

# Utilisation de la méthode Louvain 

In [ ]:
louvain = Louvain().fit(A)

In [ ]:
modularity(A,louvain.labels_)

In [ ]:
np.unique(louvain.labels_).shape[0] #Nb de clusters

#### Représentation graphique du graphe et des clusters

In [ ]:
SVG(svg_digraph(A,labels=louvain.labels_))

# Génération de graphes pour la visualisation des clusters

In [ ]:
def graphViz(A, nodeType="Node", relType="LINKED_TO", version="V1", use_weights=False):
    """
    Crée le script d'un graphe permettant de visualiser les différents clusters donnés 
    par la méthode Louvain
    """
    
    louvain = Louvain().fit(A)
    
    with open(f"L{relType}{version}{str_weight}.cypher","w") as out:
        #Récupération des ids et noms de tous les noeuds
        nodeQuery = f"""
        {query_start(same_ns=type_same_ns)} RETURN DISTINCT ID(n1) AS idNode, n1.{q_name()} AS nodeName
        """
        nodes = session.run(nodeQuery)


        for i,node in enumerate(nodes):
            idNode, nodeName = new_node_id(node["idNode"]), node["nodeName"]
            cluster_label = louvain.labels_[idNode]
            
            #Chaque noeud a pour type le label du cluster qui lui a été attribué par le clustering
            out.write(f'CREATE (n{idNode+1}:Cluster{cluster_label+1}{{name:"{nodeName}"}})\n')

        relationQuery = f"""
        {query_start("r",type_same_ns)} RETURN ID(n1) AS id1, ID(n2) AS id2, TOINTEGER(r.{q_poids()}) AS poids, r.optional AS opt
        """
        relations = session.run(relationQuery)

        #On recrée les relations entre les noeuds
        for relation in relations:
            id1, id2 = new_node_id(relation["id1"]), new_node_id(relation["id2"])
            cluster_label = louvain.labels_[id1]
            #print(id1+1, id2+1)
            if relType in {"USE_NS","USE_TYPE"} and use_weights:
                w = (relation["poids"]*(0.1 if relation["opt"] == "yes" else 1)+1)/(max_poids+1)
                out.write(f'CREATE (n{id1+1})-[:{relType}{{poids:{round(w,3)}}}]->(n{id2+1})\n')
            else:
                out.write(f'CREATE (n{id1+1})-[:{relType}]->(n{id2+1})\n')
        out.write(";")

In [ ]:
def graphVizAbstract(A, nodeType="Node", relType="LINKED_TO", version="V1", use_weights=False):
    """
    Crée le script d'un graphe pour visualiser le clustering obtenu 
    avec la méthode Louvain de manière plus abstraite
    """
    
    louvain = Louvain().fit(A)
    k = np.unique(louvain.labels_).shape[0]
    
    
    A_k = np.zeros((k,k))
    
    names_per_cluster = {i+1:[] for i in np.unique(louvain.labels_)}

    id_counter = 0
    

    
    with open(f"L{relType}Abs{version}{str_weight}.cypher","w") as out:
        #Récupération des ids et noms de tous les noeuds
        nodeQuery = f'{query_start(same_ns=type_same_ns)} RETURN DISTINCT ID(n1) AS idNode, n1.{q_name()} AS nodeName'
        nodes = session.run(nodeQuery)
        
        for node in nodes:
            idNode, nodeName = new_node_id(node["idNode"]), node["nodeName"]
            label = louvain.labels_[idNode]
            names_per_cluster[label+1].append(nodeName)

        #Récupération des relations avec leur poids et son optionalité si présents
        relationQuery = f"""
        {query_start("r",type_same_ns)} RETURN ID(n1) AS id1, ID(n2) AS id2, TOINTEGER(r.{q_poids()}) AS poids, r.optional AS opt
        """
        relations = session.run(relationQuery)

        #Calcul de la matrice d'adjacence entre clusters
        for relation in relations:
            id1, id2 = new_node_id(relation["id1"]), new_node_id(relation["id2"])
            label1, label2 = louvain.labels_[id1], louvain.labels_[id2]
            if label1 != label2: #On ne compte que les relations entre noeuds de clusters différents
                if relType in {"USE_NS","USE_TYPE"} and use_weights:
                    A_k[label1,label2] += (relation["poids"]*(1 if relation['opt'] == "no" else 0.1)+1)/(max_poids+1)
                else:
                    A_k[label1,label2] += 1

        #Création des noeuds
        for i in range(k):
            out.write(f'CREATE (n{i+1}:Cluster{i+1}{{name:"C{i+1}",names:{names_per_cluster[i+1]}, nbNoeuds:{len(names_per_cluster[i+1])}}})\n')

        #Création des relations entre clusters
        for ci,cluster in enumerate(A_k):
            for i,w in enumerate(cluster):
                #print(i+1, w)
                if w != 0:
                    out.write(f'CREATE (n{ci+1})-[:{relType}{{poids:{round(w,3)})}}]->(n{i+1})\n')

        out.write(";")
        #print(A_k)
        #print(names_per_cluster)
    

In [ ]:
graphViz(A, nodeType, relType, version, use_weights)

In [ ]:
graphVizAbstract(A,nodeType, relType, version, use_weights)

------------------------